In [2]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import copy
import os
import xgboost as xgb
import seaborn as sns
import xarray as xr
import sys
sys.path.append("/home/ac.tzhang/my_utils/src/")
from plot import contour
import cartopy.crs as ccrs
from global_land_mask import globe

In [3]:
from cycler import cycler
my_cycler = (cycler(color=['#94c8d8','#FE9A84','#296540','#E63F39','#C3AAD1',
              '#D5AC5A','#F4B6C3','#B4BCCA','#DBD468','#B1D3E1','#8b1821']))

fh = 13
plt.rcParams['figure.figsize'] = (18,12)
plt.rcParams['axes.prop_cycle'] = my_cycler
plt.rcParams['lines.linewidth'] = 3
plt.rcParams['lines.markersize'] = 9#12
plt.rcParams['legend.fontsize'] = fh
plt.rcParams['axes.labelsize'] = fh
plt.rcParams['xtick.labelsize'] = fh
plt.rcParams['ytick.labelsize'] = fh
plt.rcParams['axes.titlesize'] = fh
plt.rcParams['figure.titlesize'] = fh

In [97]:
class diags():
    def __init__(self):
        self.varns = ['SWCF global ceres_ebaf_toa_v4.1','LWCF global ceres_ebaf_toa_v4.1','PRECT global GPCP_v2.3',
           'FLNS global ceres_ebaf_surface_v4.1','U-200mb global ERA5','PSL global ERA5','Z3-500mb global ERA5',
           'TREFHT land ERA5','T-200mb global ERA5','SST global HadISST_PI']
        self.shortname = ['SWCF','LWCF','PRECT','FLNS','U200','PSL','Z500','TREFHT','T200','SST']
        self.const = 'RESTOM global ceres_ebaf_toa_v4.1'
        self.const_shortname = 'Net TOA'
        self.all_varns = ['PRECT global GPCP_v2.3','FLUT global ceres_ebaf_toa_v4.1','FSNTOA global ceres_ebaf_toa_v4.1',
                          'SWCF global ceres_ebaf_toa_v4.1','LWCF global ceres_ebaf_toa_v4.1','NETCF global ceres_ebaf_toa_v4.1',
                          'FLNS global ceres_ebaf_surface_v4.1','FSNS global ceres_ebaf_surface_v4.1','LHFLX global ERA5',
                          'SHFLX global ERA5','PSL global ERA5','U-850mb global ERA5','U-200mb global ERA5',
                          'Z3-500mb global ERA5','T-850mb global ERA5','T-200mb global ERA5',
                          'TAUXY ocean ERA5','TREFHT land ERA5','SST global HadISST_PI']
        self.all_shortname = ['PRECT','FLUT','FSNTOA','SWCF','LWCF','NETCF','FLNS','FSNS','LHFLX','SHFLX',
                              'PSL','U850','U200','Z500','T850','T200','TAUXY','TREFHT','SST_PI']
        self.key_varns = ['SWCF global ceres_ebaf_toa_v4.1','LWCF global ceres_ebaf_toa_v4.1',
                          'PRECT global GPCP_v2.3','TREFHT land ERA5','PSL global ERA5','U-200mb global ERA5','U-850mb global ERA5',
                          'Z3-500mb global ERA5']
        self.key_shortname = ['SW CRE','LW CRE','prec','tas land','SLP','u-200','u-850','Zg-500']

        self.p_names = ['clubb_c1','clubb_gamma_coef','clubb_c_k10','zmconv_tau','zmconv_dmpdz', 'zmconv_micro_dcs',
            'nucleate_ice_subgrid','p3_nc_autocon_expon','p3_qc_accret_expon','zmconv_auto_fac',
            'zmconv_accr_fac','zmconv_ke','cldfrc_dp1','p3_embryonic_rain_size','effgw_oro']

        self.path_base = f"/home/ac.tzhang/fs0_large/E3SM_archive/"
        self.path_base = f'/home/ac.wlin/www/E3SMv3_dev/20231213.v3.LR.piControl-PPE.chrysalis/'
        self.path_diag = f"/e3sm_diags/atm_monthly_180x360_aave/model_vs_obs_0101-0110/viewer/table-data/ANN_metrics_table.csv"

        self.get_cntl_score()
    
    def get_cntl_score(self):
        self.cntl_name = f'20231209.v3.LR.piControl-spinup.chrysalis'
        data_diags = pd.read_csv(f'{self.path_base}/{self.cntl_name}/{self.path_diag}').set_index('Variables')
        data_diags = data_diags.loc[self.key_varns]
        self.rmse_cntl = data_diags['RMSE']

    def get_score(self,case_name):
        data_diags = pd.read_csv(f'{self.path_base}/{case_name}/{self.path_diag}').set_index('Variables')
        const = data_diags['Test_mean'].loc[self.const]
        data_diags = data_diags.loc[self.key_varns]
        score = data_diags['RMSE']/self.rmse_cntl
        score.columns = self.key_shortname
        score['metrics'] = np.sum(score.values) + const
        return score

    def get_scoreset(self):
        self.scoreset = pd.DataFrame()
        case_prefix = '20231213.v3.LR.piControl-PPE.chrysalis'
        for i in range(1,29):
            cn = f'{case_prefix}_M{i:03d}'
            tmp = self.get_score(cn)
            self.scoreset[f'M{i:03d}'] = tmp

        self.scoreset = self.scoreset.T
        self.scoreset.columns = self.key_shortname + ['metrics']
        return self.scoreset

In [98]:
dd = diags()
a = dd.get_scoreset()

In [99]:
a

,SW CRE,LW CRE,prec,tas land,SLP,u-200,u-850,Zg-500,metrics
M001,0.954848,0.944670,1.016211,0.915038,0.910765,0.985525,1.013449,0.805556,7.945062
M002,0.953767,0.973496,1.015198,0.927782,0.908719,1.056360,1.053006,0.841270,8.626597
M003,0.970980,0.997485,1.015198,0.962617,0.945149,0.954111,1.004747,0.845238,7.837525
M004,0.967986,0.960340,1.006079,0.963042,0.882112,1.020634,1.009494,0.841270,7.849957
M005,0.975636,1.056878,1.020263,0.987256,0.941875,1.004620,1.032437,0.825397,8.002361
M006,0.955596,1.004063,1.023303,0.957519,0.922636,0.938097,1.004747,0.813492,8.023453
M007,0.988275,1.067324,1.028369,1.000425,1.018829,0.942100,0.977848,0.817460,7.751632
M008,0.957176,0.961308,0.966565,0.971113,1.013099,0.928549,0.974684,0.849206,7.828700
M009,0.989024,1.031728,1.004053,0.988105,1.022923,0.950416,0.973101,0.888889,7.986238
M010,0.976551,1.010834,0.988855,0.985981,0.955792,0.983061,1.018987,0.916667,8.112728
